This notebook uses the Scientific Python (scipy) stack tools to generate flow duration curves from current USGS NWIS data.

Using recipes from this notebook, you can make:
* USGS Station Summaries
* Flow duration curves
* Iterative import and compilation of USGS station information and data
* boxplots using pandas
* iterative charts (one monthly summary boxplot per station)
* Gantt charts of USGS stations

## Background

Check out this for some great `pandas` applications:
http://earthpy.org/time_series_analysis_with_pandas_part_2.html

In [1]:
%matplotlib inline
import pandas as pd
import urllib2
from pyproj import Proj, transform
import xmltodict
import numpy as np
from datetime import datetime, date, timedelta
import mechanize

In [2]:
huc = '16010203'
stationhtml = "http://waterservices.usgs.gov/nwis/site/?format=rdb,1.0&huc="+str(huc)+"&siteType=GW&hasDataTypeCd=gw"
response = urllib2.urlopen(stationhtml)
html = response.read()
skip = html[:html.rfind('#\n')+2].count('\n')
skiplist = range(0,skip)
skiplist.append(skip+1)
sites = pd.read_table(stationhtml,sep="\t",skiprows=skiplist)

In [3]:
stations = list(sites['site_no'].values)
stations = [str(i) for i in stations]
stations = ', '.join(stations)
stations = stations.replace(" ","")

In [4]:
stationinfohtml = "http://waterservices.usgs.gov/nwis/site/?format=rdb&sites="+stations+"&siteOutput=expanded"
response = urllib2.urlopen(stationinfohtml)
html = response.read()
skip = html[:html.rfind('#\n')+2].count('\n')
skiplist = range(0,skip)
skiplist.append(skip+1)
siteinfo = pd.read_table(stationinfohtml, sep="\t",skiprows=skiplist)

In [5]:
siteinfo.columns

Index([u'agency_cd', u'site_no', u'station_nm', u'site_tp_cd', u'lat_va',
       u'long_va', u'dec_lat_va', u'dec_long_va', u'coord_meth_cd',
       u'coord_acy_cd', u'coord_datum_cd', u'dec_coord_datum_cd',
       u'district_cd', u'state_cd', u'county_cd', u'country_cd',
       u'land_net_ds', u'map_nm', u'map_scale_fc', u'alt_va', u'alt_meth_cd',
       u'alt_acy_va', u'alt_datum_cd', u'huc_cd', u'basin_cd', u'topo_cd',
       u'instruments_cd', u'construction_dt', u'inventory_dt',
       u'drain_area_va', u'contrib_drain_area_va', u'tz_cd', u'local_time_fg',
       u'reliability_cd', u'gw_file_cd', u'nat_aqfr_cd', u'aqfr_cd',
       u'aqfr_type_cd', u'well_depth_va', u'hole_depth_va', u'depth_src_cd',
       u'project_no'],
      dtype='object')

In [6]:
def getelev(x):
    elev = "http://ned.usgs.gov/epqs/pqs.php?x="+str(x[0])+"&y="+str(x[1])+"&units=Meters&output=xml"
    response = urllib2.urlopen(elev)
    html = response.read()
    d = xmltodict.parse(html)
    return float(d['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

In [7]:
def getwlelev(x):
    return x[1] - (x[0]/3.2808)

In [8]:
def qqq(x):
    x.rstrip().lstrip()
    j = x.split(' ')
    a = j[0][:1]
    b = j[0][1:]
    c = j[1][:1]
    d = j[1][1:]
    e = [a,b,c,d,j[2],j[3],j[4],j[5],j[6]]
    
    NS = int(e[1].replace(',',''))
    EW = int(e[3].replace(',',''))
    qc = e[4]
    d1 = e[0]
    d2 = e[2]
    dic1 = {'NE':'a','NW':'b','SW':'c','SE':'d'}
    qcdDict = {'E4S':'d','E4N':'a','N4E':'a','N4W':'b','W4N':'b','W4S':'c','S4W':'c','S4E':'d'}
    dic2 = {'a':'b','b':'a','c':'d','d':'c'}
    dic3 = {'a':'d','b':'c','c':'b','d':'a'}
    dic4 = {'a':'c','b':'d','c':'a','d':'b'}
    if qc[-1]=='4':
        if qc[0]=='N' or qc[0]=='S':
            qcd = qc+d2
        elif qc[0]=='E' or qc[0]=='W':
            qcd = qc+d1
        q1 = qcdDict.get(qcd,'x')
    elif qc in ('NE','NW','SW','SE'):
        q1 = dic1.get(qc)
    else:
        print "invalid quarter"
        q1 = 'X'
    if NS < 1320:
        if EW <1320:
            q2 = q1
        elif EW >1320:
            qd2 = {'a':'b','b':'a','c':'d','d':'c'}
            q2 = dic2.get(q1,'x')
    elif NS > 1320:
        if EW <1320:
            q2 = dic3.get(q1,'x')
        elif EW >1320:
            q2 = dic4.get(q1,'x')
    else:
        q2 = 'X'

    if NS < 660 or (NS > 1320 and NS < 1980):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = q1
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic2.get(q1,'x')
    elif (NS > 660 and NS < 1320) or (NS > 1980 and NS < 2640):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = dic3.get(q1,'x')
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic4.get(q1,'x')
    else:
        q3 = 'X'
    Tn = e[6][:-1].rjust(2)
    Rn = e[7][:-1].rjust(2)
    Sec = e[5].rjust(2)
    TRd = e[6][-1]+e[7][-1]
    TR = dic1.get(TRd).upper()
    CAD = '('+TR+'-'+Tn+'-'+Rn+')'+Sec+q1+q2+q3+'-1'
    return CAD                

In [9]:
def getPLSS(desc):
    PLSStitlen = len("PLSS Description is </font><font size='4'> ")
    PLSSbeg = desc.find("PLSS Description is </font><font size='4'>")+PLSStitlen
    PLSSend = desc[PLSSbeg:].find(" <br> ")+PLSSbeg
    PLSSdesc = desc[PLSSbeg:PLSSend]
    QRTRend = desc.find(" of the above Section </font></center><p>")
    QRTRbeg = desc.find("The point is found in the ")+len("The point is found in the ")
    QRTRdesc = desc[QRTRbeg:QRTRend]
    SecBeg = PLSSdesc.find("Section ")+len("Section ")
    SecEnd = PLSSdesc.find(", Township ")
    Section = int(PLSSdesc[SecBeg:SecEnd])
    TownBeg = SecEnd + len(", Township ")
    TownEnd = PLSSdesc.find(", Range ")
    Township = PLSSdesc[TownBeg:TownEnd]
    RangeBeg = TownEnd + len(", Range ")
    RangeEnd = RangeBeg + PLSSdesc[RangeBeg:].find(", ")
    Range = PLSSdesc[RangeBeg:RangeEnd]

    PLSS = PLSSdesc.replace("South ","S")
    PLSS = PLSS.replace("West ","W")
    PLSS = PLSS.replace("North ","N")
    PLSS = PLSS.replace("East ","E")
    PLSS = PLSS.replace("feet ","")
    PLSS = PLSS.replace(" and "," ")
    PLSS = PLSS.replace("from the ","")
    BM = PLSS[-6:-4]
    PLSScn = PLSS.find(" Corner")
    PLSS = PLSS[:PLSScn]
    PLSS = PLSS.replace(',','')
    PLSS = PLSS + " " +str(Section) + " " + Township + " " + Range + " " + BM
    
    try:
        CAD = qqq(PLSS)
    except(AttributeError,TypeError,UnboundLocalError):
        print PLSS
        CAD = np.nan
    return PLSS, CAD

In [10]:
PLSS = 'N337 E1,117 W4 3 9N 1E SL'
print qqq(PLSS)

(A- 9- 1) 3bba-1


In [11]:
def proj(x):
    inProj = Proj(init='epsg:4326') #WGS84
    outProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2, y2

siteinfo['UTM_X'], siteinfo['UTM_Y'] = zip(*siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: proj(x),1))

In [12]:
def USGSID(x):
    def dms(dec):
        DD = str(int(abs(dec)))
        MM = str(int((abs(dec) - int(DD))*60)).zfill(2)
        SS = str(int(round((((abs(dec) - int(DD))*60) - int(MM))*60, 0))).zfill(2)
        return DD+MM+SS
    return 'UT'+dms(x[1])+dms(x[0])+'01'

In [13]:
def getwellinfo(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS, CAD

http://wwwsearch.sourceforge.net/mechanize/forms.html

In [112]:
def winmatch(x):
    request = mechanize.Request("http://waterrights.utah.gov/wellinfo/wellsearch.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0] 
    #print form
    form["mainoption"]=["radius"]
    form["SearchRadius"]="2000"
    form["option"]=["UTM"]
    form["xUTM"]=str(x[0])
    form["yUTM"]=str(x[1])
    if np.isfinite(x[2]):
        form["DateRange"]=["on"]
        form["edtDrillBef"]=str(int(str(x[2])[:4])+5)
        form["edtDrillAft"]=str(int(str(x[2])[:4])-5)
        #print str(int(str(x[2])[:4]))
    if np.isfinite(x[3]):
        form["DepthRange"]=["on"]
        form["edtDrillBel"]=str(int(x[3])-10)
        form["edtDrillAbo"]=str(int(x[3])+30)
        #print str(str(int(x[3])))
    win =  mechanize.urlopen(form.click()).read()
    winbeg = win.find('WIN=')
    if winbeg == -1:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan 
    else:
        wintabeg=win.find('<table',win.find('<table')+5)
        wintaend=win.find('</table>')
        winmatches = pd.read_html(win[wintabeg:wintaend], header=0, skiprows=0)
        winmatches = winmatches[0]
        winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
        winmatches.rename(columns=winDic,inplace=True)
        return winmatches.ix[0,0], winmatches.ix[0,1], winmatches.ix[0,2], winmatches.ix[0,3], winmatches.ix[0,4], winmatches.ix[0,5], int(winmatches.ix[0,6])

In [77]:
winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
winmatches.rename(columns=winDic,inplace=True)
list(winmatches.ix[0])

[u'25-10783',
 882,
 155.0,
 155.0,
 u'08/25/1967',
 u'S 975 W 4572 NE 28 10N 1E SL',
 432043,
 nan]

In [15]:
siteinfo['Elev'] = siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: getelev(x),1)

In [16]:
siteinfo['USGSid'] = siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: USGSID(x),1)

In [17]:
siteinfo['PLSS'], siteinfo['CAD'] = zip(*siteinfo[['UTM_X','UTM_Y']].apply(lambda x: getwellinfo(x),1))

N89 E1320 S4 31 12N 1E SL


In [113]:
siteinfo['WRNUM'], siteinfo['Dist'], siteinfo['Diam'], siteinfo['DepthWR'], siteinfo['DrillDate'], siteinfo['Loc'], siteinfo['WIN'] = zip(*siteinfo[['UTM_X','UTM_Y','construction_dt','well_depth_va']].apply(lambda x: winmatch(x),1))

In [114]:
siteinfo.to_clipboard()

In [22]:
datahtml = "http://waterservices.usgs.gov/nwis/gwlevels/?format=rdb&sites="+stations+"&startDT=1800-01-01&endDT="+str(datetime.today().year)+"-"+str(datetime.today().month).zfill(2)+"-"+str(datetime.today().day).zfill(2)
response = urllib2.urlopen(datahtml)
html = response.read()
skip = html[:html.rfind('#\n')+2].count('\n')
skiplist = range(0,skip)
skiplist.append(skip+1)
data = pd.read_table(datahtml, sep="\t",skiprows=skiplist)

In [23]:
data.drop([u'agency_cd', u'site_tp_cd'], inplace=True, axis=1)

In [24]:
stationWL = pd.merge(data, siteinfo, on='site_no', how='left')

In [25]:
stationWL['dec_coord_datum_cd'].unique()

array(['NAD83'], dtype=object)

In [26]:
stationWL = stationWL[~stationWL['lev_status_cd'].isin(['Z', 'R', 'V', 'P', 'O', 'F', 'W', 'G', 'S', 'C', 'E', 'N'])]

In [27]:
stationWL['wlelev'] = stationWL[['lev_va','Elev']].apply(lambda x: getwlelev(x),1)

In [29]:
z = stationWL.wlelev.values
x = stationWL.UTM_X.values
y = stationWL.UTM_Y.values

http://connor-johnson.com/2014/03/20/simple-kriging-in-python/

http://stackoverflow.com/questions/31124930/ckdtree-vs-dsearchn?lq=1

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
A = np.loadtxt('A.txt')
B = np.loadtxt('B.txt')
tree = cKDTree( B[:,[1,2,3]] )
d, inds = tree.query( A[:,[1,2,3]], k=1, p=2)
B_new = B[inds]
xyz_near = np.hstack(( B_new[:,0:4], A[:,0:4] ))

for j, a in enumerate(A):
    # compute 2-norms from each point in B to a
    dd = np.sqrt(((a[1:] - B[:,1:])**2).sum(axis=1))
    # find closest point
    jx = np.argmin(dd)
    # check solution
    assert inds[j] == jx
    assert np.allclose(d[j], dd.min())
    # check it is unique
    assert (dd[jx+1:] > d[j]).all()
    assert (dd[:jx] > d[j]).all()

print("All OK")

## Visualizing Data

## Exporting Figures

The following script will generate a series of box and whisker plots and save them in a pdf. It makes a box plot for each station, breaking the data into monthly intervals.  Make sure to change the directory name in the script so it ends up in a recognizable place on your computer.  

In [ ]:
# create dictionary of integers and their month equivalent
months = {'1':'Jan.', '2':'Feb.', '3':'Mar.', '4':'Apr.', '5':'May', '6':'Jun.', 
         '7':'Jul.', '8':'Aug.', '9':'Sep.', '10':'Oct.', '11':'Nov.', '12':'Dec.', 'Total':'Total'}
# create empty dictionary to hold pandas Dataframes
j = {}


with PdfPages(rootname + 'station_boxplots.pdf') as pdfs:
    ymax = 10000
    ymin = 0.01
    for i in range(len(sites)):
        # make a dataframe containing summary statistics and store it in the j dictionary
        j[sites[i]] = USGS_Site_Data.groupby('mon')[sites[i]].agg({'min':np.min, 'mean':np.mean, 
                                                                   'median':np.median, 'max':np.max, 'std':np.std, 
                                                                   'cnt':(lambda x: np.count_nonzero(~np.isnan(x)))}).reset_index()
        # make a list of the custom lables you will use for your boxplot; this one will show the number of samples used to make the plot
        labs = [months[(str(j[sites[i]]['mon'][b]))] + " (n=" + str(int(j[sites[i]]['cnt'][b])) + ")" for b in range(len(j[sites[i]]))]
        # designate the location of each custom label
        tickloc = [b+1 for b in range(len(j[sites[i]]['mon']))]
        
        plt.figure()
        USGS_Site_Data.boxplot(column=sites[i],by='mon', rot=70)
        strtdt = str(USGS_Site_Info.ix[sites[i],'start_date'])[0:10]
        findt = str(USGS_Site_Info.ix[sites[i],'fin_date'])[0:10]
        siteName = USGS_Site_Info.ix[sites[i],'name'].title() 
        plt.title( siteName + ' (' + sites[i] + ')  ' + strtdt + ' to ' + findt )
        plt.suptitle('')
        plt.yscale('log')
        plt.ylabel('Discharge (cfs)')
        plt.ylim((ymin,ymax))
        plt.xlabel('Month')
        # here is where your lists for the custom label come into play
        plt.xticks(tickloc, labs)
        
        pdfs.savefig()

        plt.close()
    # Save metadata of the pdf so you can find it later
    d = pdfs.infodict()
    d['Title'] = 'Monthly Station USGS Boxplots UMSS'
    d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
    d['Subject'] = 'Boxplots of several USGS Surface Stations'
    d['Keywords'] = 'USGS Surface NWIS Boxplot'
    d['CreationDate'] = datetime.today()
    d['ModDate'] = datetime.today()

Let's plot a few of the boxplots so you can see what they look like.

In [ ]:
for i in range(1,3):
    j[sites[i]] = USGS_Site_Data.groupby('mon')[sites[i]].agg([np.min, np.mean, np.median, np.max, np.std, np.size]).reset_index()
    # make a list of the custom lables you will use for your boxplot; this one will show the number of samples used to make the plot
    labs = [months[(str(j[sites[i]]['mon'][b]))] + " (n=" + str(int(j[sites[i]]['size'][b])) + ")" for b in range(len(j[sites[i]]))]
    # designate the location of each custom label
    tickloc = [b+1 for b in range(len(j[sites[i]]['mon']))]

    plt.figure()
    USGS_Site_Data.boxplot(column=sites[i],by='mon', rot=70)
    plt.title(USGS_Site_Info.ix[sites[i],'name'].title() + ' (' + sites[i] + ')')
    plt.suptitle('')
    plt.yscale('log')
    plt.ylabel('Discharge (cfs)')
    plt.ylim((ymin,ymax))
    plt.xlabel('Month')
    # here is where your lists for the custom label come into play
    plt.xticks(tickloc, labs)
    plt.show()
    plt.close()

This script will generate boxplots showing all of the station data.

In [ ]:
# This script summarizes discharge for all sites and limits the number of box plots on one graph to the n variable
j=0
with PdfPages(rootname + 'sum_boxplots.pdf') as pdf:
    while j < len(sites):
        ymax = 10000
        ymin = 0.01
        n=10
        # if statement allows for uneven number of sites on last page
        if j+n >= len(sites):
            plt.figure()
            USGS_Site_Data[sites[j:-1]].plot(kind='box')
            plt.title('Sites '+sites[j]+' to '+sites[-1] )
            plt.yscale('log')
            plt.xlabel('USGS Site')
            plt.xticks(rotation=45)
            plt.ylabel('discharge (cfs)')
            plt.ylim((ymin,ymax))
            pdf.savefig()
            plt.show()
            plt.close()
            j = j+n
        else:
            plt.figure()
            USGS_Site_Data[sites[j:j+n]].plot(kind='box')
            plt.title('Sites '+sites[j]+' to '+sites[j+n] )
            plt.yscale('log')
            plt.xlabel('USGS Site')
            plt.xticks(rotation=45)
            plt.ylabel('discharge (cfs)')
            plt.ylim((ymin,ymax))
            pdf.savefig()
            plt.show()
            plt.close()
            j = j+n
        # Save metadata of the pdf so you can find it later
        d = pdf.infodict()
        d['Title'] = 'Summary USGS Boxplots UMSS'
        d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
        d['Subject'] = 'Boxplots of several USGS Surface Stations'
        d['Keywords'] = 'USGS Surface NWIS Boxplot'
        d['CreationDate'] = datetime.today()
        d['ModDate'] = datetime.today()

We should also produce hydrographs of each station.

In [ ]:
xmax = USGS_Site_Data.index.astype(datetime).values[-1]
xmin = USGS_Site_Data.index.astype(datetime).values[0]

pdfs = PdfPages(rootname + 'station_hydrographs.pdf')
ymax = 10000
ymin = 0.1
for i in range(len(sites)):
    x = USGS_Site_Data.index.values
    y = USGS_Site_Data[sites[i]].values
    plt.figure()
    plt.plot(x,y)
    strtdt = str(USGS_Site_Info.ix[sites[i],'start_date'])[0:10]
    findt = str(USGS_Site_Info.ix[sites[i],'fin_date'])[0:10]
    siteName = USGS_Site_Info.ix[sites[i],'name'].title() 
    plt.title( siteName + ' (' + sites[i] + ')  ' + strtdt + ' to ' + findt )
    plt.suptitle('')
    plt.yscale('log')
    plt.ylabel('Discharge (cfs)')
    plt.ylim((ymin,ymax))
    plt.xlabel('Year')
    plt.xticks(np.arange(datetime(1905,1,1),xmax+timedelta(days=365.25),timedelta(days=365.25*5)),rotation=45)
    plt.xlim(xmin,xmax)
    pdfs.savefig()
    plt.close()
    # Save metadata of the pdf so you can find it later

d = pdfs.infodict()
d['Title'] = 'Monthly Station USGS Hydrographs UMSS'
d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
d['Subject'] = 'Hydrograph of several USGS Surface Stations'
d['Keywords'] = 'USGS Surface NWIS Hydrograph'
d['CreationDate'] = datetime.today()
d['ModDate'] = datetime.today()

In [ ]:
pd.date_range(start=xmin, end=xmax, freq='5AS').year

In [ ]:
xmax = USGS_Site_Data.index[-1]
xmin = USGS_Site_Data.index[0]

plt.figure()
ticks = pd.date_range(start=xmin, end=xmax, freq='4AS')
USGS_Site_Data[sites[0:3]].plot(subplots=True,sharex=True,figsize=(10,8),logy=True, rot=90)
plt.xlim(xmin,xmax)
labs = pd.date_range(start=xmin, end=xmax, freq='4AS').year
plt.xticks(ticks,labs)
plt.show()
plt.close()

In [ ]:
def lumped_hydro(i1,i2):
    pdfs = PdfPages(rootname + 'station_hydrographs_lumped.pdf')
    plt.figure()
    ticks = pd.date_range(start=xmin, end=xmax, freq='4AS')
    USGS_Site_Data[sites[i1:i2]].plot(subplots=True, sharex=True, figsize=(10,24),logy=True, rot=90)
    plt.xlim(xmin,xmax)
    labs = pd.date_range(start=xmin, end=xmax, freq='4AS').year
    plt.xticks(ticks,labs)
    pdfs.savefig()
    plt.close()

In [ ]:
lumped_hydro(0,10)
lumped_hydro(10,20)

In [ ]:
lumped_hydro(20,30)
lumped_hydro(30,-1)

This script will iteratively produce Flow Duration Curves for each of the stations. A flow duration curve is a <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#PPF>percent point function (ppf)</a>, displaying discharge as a function of probability of that discharge occuring. The ppf is the inverse of the better known <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#CDF>cumulative distribution function (cdf)</a>. See <a href=http://pubs.usgs.gov/wsp/1542a/report.pdf>this USGS publication</a> for more information.

In [ ]:
with PdfPages(rootname+'station_fdc.pdf') as pdf:
    ymax = 10000
    ymin = 0.01
    for i in range(len(sites)):
        plt.figure()
        fdc_simple(USGS_Site_Data,sites[i],1900,2015)
        fdc_simple(USGS_Site_Data,sites[i],1900,1970)
        fdc_simple(USGS_Site_Data,sites[i],1970,2015)
        plt.ylim(0.01,10000)
        plt.xlim(-.05,1.05)
        plt.grid(which = 'both')
        plt.legend()
        plt.xlabel('probability that discharge was exceeded or equaled')
        plt.title('Flow duration curve for ' + str(USGS_Site_Info['name'][i]).title() + ' ('+ sites[i] +')'+'\n'+
                  'Record: ' + str(USGS_Site_Info['start_date'][i])[0:10] + ' to ' + str(USGS_Site_Info['fin_date'][i])[0:10])
        plt.yscale('log')
        plt.ylabel('discharge (cfs)')
        plt.xticks(np.arange(0,1.05,0.05))
        pdf.savefig()
        plt.close()
    # Save metadata of the pdf so you can find it later
    d = pdf.infodict()
    d['Title'] = 'Flow Duration Curves USGS'
    d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
    d['Subject'] = 'Flow Duration Curves of several USGS Surface Stations'
    d['Keywords'] = 'USGS Surface NWIS FDC Flow Duration'
    d['CreationDate'] = datetime.today()
    d['ModDate'] = datetime.today()

For brevity, here is an example plot from the output of the script above.

In [ ]:
plt.figure()
fdc_simple(USGS_Site_Data,sites[38],1900,2015)
fdc_simple(USGS_Site_Data,sites[38],1900,1970)
fdc_simple(USGS_Site_Data,sites[38],1970,2015)

plt.yscale('log')
plt.grid(which = 'both')
plt.xlabel('% of time that indicated discharge was exceeded or equaled')
plt.ylabel('discharge (cfs)')
plt.xticks(np.arange(0.0,1.05,0.05))
plt.title('Flow duration curve for ' + str(USGS_Site_Info['name'][38]) + ' ('+ sites[i] +')'+'\n'+ 
          'Record: ' + str(USGS_Site_Info['start_date'][i])[0:10] + ' to ' + str(USGS_Site_Info['fin_date'][i])[0:10])
plt.legend()

Below are some sad attempts to model the flow duration curves:

In [ ]:
site = sites[28]
df = USGS_Site_Data[[site]]
begyear = 1900
endyear = 2015
data = df[(df.index.to_datetime() > pd.datetime(begyear,1,1))&(df.index.to_datetime() < pd.datetime(endyear,1,1))]
data = data.dropna()

data['doy']=data.index.dayofyear
dailyavg = data[site].groupby(data['doy']).mean()


data = np.sort(dailyavg)

mean = np.mean(data)
std = np.std(data)
f = [(data[i]) for i in range(len(data))]
#f = [(data[i])/mean for i in range(len(data))]
#f = [(data[i]-std)/mean for i in range(len(data))]

ranks = sp.rankdata(f, method='average')
ranks = ranks[::-1]
prob = [(ranks[i]/(len(f)+1)) for i in range(len(f)) ]

x = prob
y = f

plt.figure()
plt.scatter(y,x,label=site,color='blue')
#plt.yscale('log')
#plt.xlim(-.05,1.05)
plt.grid(which = 'both')
plt.ylabel('probability that discharge was exceeded or equaled')
plt.xlabel('normalized discharge')
#plt.xticks(np.arange(0,1.00,0.05))
plt.title('Flow duration curve for ' + site + ' averaged from ' + str(begyear) + ' to ' + str(endyear))

def func(x,a,b,c,d):
    return a*np.log(x*b+c)+d

par, cov = op.curve_fit(func,y,x,p0=[-0.16528617, 1.54535185, -24.70440088, 0.9])
plt.plot(y, [par[0]*np.log(y[i]*par[1]+par[2])+par[3] for i in range(len(y))], color='red')
print 'curve fit', sp.linregress(x,[par[0]*np.log(y[i]*par[1]+par[2])+par[3] for i in range(len(y))])

plt.figure()
plt.scatter(x,y,label=site,color='blue')
#plt.yscale('log')
#plt.xlim(-.05,1.05)
plt.grid(which = 'both')
plt.xlabel('probability that discharge was exceeded or equaled')
plt.ylabel('normalized discharge')
#plt.xticks(np.arange(0,1.00,0.05))
plt.title('Flow duration curve for ' + site + ' averaged from ' + str(begyear) + ' to ' + str(endyear))

def func2(x,a,b,c,d):
    return a*np.exp(x*b+c)+d


parm, covm = op.curve_fit(func2,x,y,p0=[-0.16528617, 0.02, 0.70440088, 0.9])
plt.plot(x, [parm[0]*np.exp(x[i]*parm[1]+parm[2])+parm[3] for i in range(len(x))], color='red')
print 'curve fit', sp.linregress(y,[parm[0]*np.exp(x[i]*parm[1]+parm[2])+parm[3] for i in range(len(x))])

In [ ]:
def dic2df(dic,head):
    df = pd.DataFrame(data=dic)
    df = df.transpose()
    df.columns = [str(head)+'_var1',str(head)+'_var2',str(head)+'_var3',str(head)+'_var4',str(head)+'_r2',str(head)+'_err']
    return df

In [ ]:
q = {}; m = {}; n = {}; k = {}; j = {}; p = {}

for site in sites:
    q[site] = fdcmatch(USGS_Site_Data,site,1900,2015,1,1)
    m[site] = fdcmatch(USGS_Site_Data,site,1900,1970,1,1)
    n[site] = fdcmatch(USGS_Site_Data,site,1970,2015,1,1)
    k[site] = fdcmatch(USGS_Site_Data,site,1900,2015,1,0)
    j[site] = fdcmatch(USGS_Site_Data,site,1900,1970,1,0)
    p[site] = fdcmatch(USGS_Site_Data,site,1970,2015,1,0)
    
dics = [q,m,n,k,j,p]
heads = ['all','to70','fm70','allin','to70in','fm70in']

USGS_q = dic2df(q,'all')
USGS_m = dic2df(m,'to70')
USGS_parms = pd.merge(USGS_q,USGS_m, left_index=True, right_index=True, how='outer' )

for i in range(2,6,1):
    x = dic2df(dics[i],heads[i])
    USGS_parms = pd.merge(USGS_parms,x, left_index=True, right_index=True, how='outer' )

USGS_parms.to_clipboard()
#USGS_finish_date = USGS_finish_date.drop([0],axis=1)
#USGS_start_fin = pd.merge(USGS_finish_date,USGS_start_date, left_index=True, right_index=True, how='outer' )

The equations from the modeled fdc plots can be used to estimate the discharge for a site based on data from a similar site.  However, the results are mediocre.

In [ ]:
n1 = 12
n2 = 11

USGS_Site_Data[sites[n1]+'p'] = [USGS_parms['all_var1'][n1]*np.log(USGS_Site_Data[sites[n1]][i]*USGS_parms['all_var2'][n1]+\
                                                                   USGS_parms['all_var3'][n1])+USGS_parms['all_var4'][n1] for i in \
range(len(USGS_Site_Data[sites[n1]]))]

USGS_Data = USGS_Site_Data[USGS_Site_Data[sites[n1]+'p']>0]

USGS_Data[sites[n2]+'d'] = [USGS_parms['allin_var1'][n2]*np.exp(USGS_Data[sites[n1]+'p'][i]*USGS_parms['allin_var2'][n2]+\
                                                                   USGS_parms['allin_var3'][n2])+USGS_parms['allin_var4'][n2] for i in \
range(len(USGS_Data[sites[n1]+'p']))]

y1 = USGS_Data[sites[n2]+'d'].values
y2 = USGS_Site_Data[sites[n2]].values

x1 = USGS_Data.index.to_datetime()
x2 = USGS_Site_Data.index.to_datetime()
y3 = USGS_Data[sites[n1]]


plt.figure()
plt.plot(x1,y1, label="modeled discharge")
plt.plot(x2,y2, label="actual discharge")
plt.plot(x1,y3, label="reference discharge")
plt.yscale('log')
plt.legend()
#plt.xlim(USGS_Site_Info['start_date'][n2],USGS_Site_Info['fin_date'][n2])
plt.xlim('1/1/1970','1/1/1974')
#plt.ylim(1,100)


http://stackoverflow.com/questions/15408371/cumulative-distribution-plots-python <br>
http://hydroclimpy.sourceforge.net/installation.html

Run the following script if you want to see a map of your stations.  This assumes that you have the <a href=http://sourceforge.net/projects/matplotlib/files/matplotlib-toolkits/>Basemap package</a> installed.

In [ ]:
from mpl_toolkits.basemap import Basemap
X = USGS_Site_Info['longitude'].astype(float).values.tolist()
Y = USGS_Site_Info['latitude'].astype(float).values.tolist()

n = 0.05 
m = Basemap(llcrnrlon=min(X)+n,llcrnrlat=min(Y)+n,urcrnrlon=max(X)+n,urcrnrlat=max(Y)+n,
            resolution='h',projection='cyl',lon_0=np.mean(X),lat_0=np.mean(Y))
m.drawrivers(color='blue',linewidth=0.5)
m.drawcounties(color='red',linewidth=0.5)
m.arcgisimage()
#m.etopo(scale=0.5)
lons = X
lats = Y
x,y = m(lons,lats)
m.plot(x,y,'ro', markersize=8)

#m.drawmapscale(lon=-114, lat=43.5, length=100, lon0=-114, lat0=39, barstyle='simple', units='km')

## Analysis in R

In [ ]:
col = USGS_Site_Data.columns

In [ ]:
import rpy2.robjects as robj
import rpy2.rlike.container as rlc

def pandas_data_frame_to_rpy2_data_frame(pDataframe):
    orderedDict = rlc.OrdDict()

    for columnName in pDataframe:
        columnValues = pDataframe[columnName].values
        filteredValues = [value if pd.notnull(value) else robj.NA_Real 
                          for value in columnValues]

        try:
            orderedDict[columnName] = robj.FloatVector(filteredValues)
        except ValueError:
            orderedDict[columnName] = robj.StrVector(filteredValues)

    rDataFrame = robj.DataFrame(orderedDict)
    rDataFrame.rownames = robj.StrVector(pDataframe.index)

    return rDataFrame

In [ ]:
USD = pandas_data_frame_to_rpy2_data_frame(USGS_Site_Data)